# 1. Overview

TF Hub 包含用于各种任务的模型；实现同一个任务的不同模型最好使用一个通用的 API ，以便于在不同模型之间进行转换；而 TF Hub 便可以实现仅仅通过改变一段字符串来交换不同模型，进而用户可以更容易地找到解决问题的最佳方案；

[Reusable SavedModel API](#2.-Reusable-SavedModels) 定义了如何将 SavedModel 加载回 Python 程序中的常规约定，以将其作为更大的 TensorFlow 模型的一部分重用；`hub.KerasLayer`类依靠`hub.load` API 将 Reusable SavedModel 打包为 Keras 层；

<br>

# 2. Reusable SavedModels

## 2.1 Introduction
SavedModel 可以通过`obj = hub.load(url)`代码加载；返回的`obj`是`tf.saved_model.load()`返回的结果，该对象可以含任意的属性，如`tf.unction`、`tf.Variables`、其他资源、以及更多递归地这样的对象；本节介绍`obj`实现的接口，以在程序中对其进行重用；符合此接口的 SavedModel 称为 Reusable SavedModel；这里重用意味着围绕`obj`构建一个更大的模型，以及对它进行微调的能力；

### 2.1.2 Relation to SignatureDefs
就`tf.function`和其他 TF2 特性而言，这个接口与 SavedModel 的 signature 是分开的；SavedModel 的 signature 可在 TF1 中使用，在 TF2 则中用于推断，如将 SavedModel 部署到 TF Serving 或 TF Lite；用于推断的 signature 表现力不足 (not expressive)，不足以支持微调，而[`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function)为重用的模型提供了更自然和更有表现力的 Python API

### 2.1.3 Relation to model-building libraries
Reusable SavedModel 只使用 TF 2 的原语，并独立于任何特定的模型构建库如 Keras 或 Sonnet；这种设定有助于不同模型构建库的重用，且不依赖原始的模型构建代码；

若需要加载 Reusable SavedModel 或将其保存到给定的模型构建库中，总需要对代码进行一些调整；对 Keras 而言，`hub.KerasLayer` 提供加载功能；Keras 内置的以 SavedModel 格式的保存方式是针对 TF2 重新设计的，以能够提供 Keras 接口的超集 (superset) (参见 2019.5 [RFC](https://github.com/tensorflow/community/blob/master/rfcs/20190509-keras-saved-model.md))

### 2.1.4 Relation to task-specific "Common SavedModel APIs"
这里的接口可以接收任意数量和类型的输入和输出；TF Hub 通用 SavedModel API 利用特定任务的使用约定来优化这个通用接口，以便于模型互换 (interchangeable)



## 2.2 Interface definition

### 2.2.1 Attributes
- `__call__`: 必须参数，用于实现模型前向传播的`tf.function`，其遵循下面所述的说明
- `variables`: `tf.Variable`对象组成的列表，任何可能调用`__call__`函数所用的变量，包括可训练和不可训练的参数；如果该列表为空，便可以将其省略；方便的是，在加载 TF1 SavedModel 来表示其`GLOBAL_VARIABLES`集合时，变量名称与`tf.saved_model.load()`综合的属性一致
- `trainable_variables`: 一系列`v.trainable=True`的`tf.Variable`对象组成的列表，其在微调时是可训练的；也可以通过设定来使某些变量在微调时固定
- `regularization_losses`: 由`tf.function`组成的列表，这些函数接受零输入并返回一个标量张量；微调时最好将其作为总损失的附加正则化项；若该列表为空，或 SavedModel 不支持微调，或不希望指定权重正则化时，可以忽略此列表

### 2.2.2 The \_\_call__ function
`outputs = obj(inputs, trainable= , ... **kwargs)`

**Args**
- inputs: 可以是
    - 单一张量
    - 张量构成的未命名的有序序列，类型为列表
    - 以特定输入名称作为键值的张量字典；
- training: 默认 Flase；如果模型支持微调，且它的计算在两者之间不同 (例如 dropout 和 BN)，则可以设置该参数为 True；It is not required that __call__ accept a Tensor-valued training argument. It falls on the caller to use tf.cond() if necessary to dispatch between them.
- kwargs: 对于张量值的参数，SavedModel creator 定义了允许的类型和形状；`tf.function`接受用`tf.TensorSpec`输入跟踪的参数的 Python 默认值；这些参数可用于自定义`__call__`中涉及的数值型超参数，例如 dropout 比例；对于 Python 值参数，SavedModel creator 定义了允许的取值，这些参数可以作为追踪函数中进行离散选择的标记；\_\_call__函数必须为所有允许的参数组合提供跟踪；翻转训练不应改变参数的被允许性；

**Result**
- 单一张量
- 张量构成的未命名的有序序列，类型为列表
- 以特定输出名称作为键值的张量字典；

返回类型可能会根据 Python 值参数的 kwargs 而有所不同，这种设定可以允许标记产生额外的输出；SavedModel creator 定义了输出类型、形状，以及它们对输入的依赖关系

由于背景知识不足，翻译水平有限，可参见[原帮助文档](https://www.tensorflow.org/hub/reusable_saved_models)

### 2.2.3 Named callables
Reusable SavedModel 可以通过生成命名子对象的方式提供多个模型片段，`obj.foo`, `obj.bar`等，每个子对象都提供一个`__call__`方法，并且支持特定于该模型片段的变量等属性；需要注意的是，这个接口不包含直接将`tf.function`添加给`tf.foo`的过程；

#  

- [Common SavedModel APIs for Image Tasks](https://www.tensorflow.org/hub/common_saved_model_apis/images)
- [make_image_classifier](https://github.com/tensorflow/hub/tree/master/tensorflow_hub/tools/make_image_classifier)
- [Retraining an Image Classifier](https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_image_retraining.ipynb)
- [Common SavedModel APIs for Text Tasks](https://www.tensorflow.org/hub/common_saved_model_apis/text)